In [74]:
import MDAnalysis as mda
from MDAnalysis.transformations import unwrap
import numpy as np
import copy
import matplotlib.pyplot as plt

# MDAnalysis unwrapping

In [75]:
u = mda.Universe("equilibrate.data", "prod.xtc")
ag = u.atoms
transform = mda.transformations.unwrap(ag)
u.trajectory.add_transformations(transform)
with mda.Writer("mda_unwrap.xtc", u.atoms.n_atoms) as W:
    for ts_temp in u.trajectory:
        W.write(u)

# Prevent fast jump

In [76]:
v = mda.Universe("equilibrate.data", "mda_unwrap.xtc")
v.transfer_to_memory()
all_positions = np.zeros((v.trajectory.n_frames, v.atoms.n_atoms, 3)) 
previous_com = np.zeros((v.residues.n_residues, 3))
com_vs_time = np.zeros((v.trajectory.n_frames, v.residues.n_residues, 3)) 
for ts in v.trajectory:
    dims = ts.dimensions[:3]
    cpt_atom = 0
    for res, ires in zip(v.residues, np.arange(v.residues.n_residues)):
        masses = res.atoms.masses
        positions = res.atoms.positions
        com = [np.sum(positions[:,0]*masses), np.sum(positions[:,1]*masses), np.sum(positions[:,2]*masses)]/np.sum(masses)
        if ts.frame > 0:
            delta_com = com - previous_com[ires]
            for delta, L, i in zip(delta_com, dims, [0, 1, 2]):
                if np.abs(delta) > L/2:
                    if delta > 0:
                        positions[:,i] -= L
                    else:
                        positions[:,i] += L
            com = [np.sum(positions[:,0]*masses),
                   np.sum(positions[:,1]*masses),
                   np.sum(positions[:,2]*masses)]/np.sum(masses)
            delta_com = com - previous_com[ires]
            for delta, L in zip(delta_com, dims):
                assert np.abs(delta) < L
            previous_com[ires] = com
        for atom_position in positions:
            all_positions[ts.frame][cpt_atom] = atom_position
            cpt_atom += 1
        com_vs_time[ts.frame][ires] = com
com_vs_time = np.array(com_vs_time)
w = mda.Universe("equilibrate.data", "mda_unwrap.xtc")
w.transfer_to_memory()
for ts_temp in w.trajectory:
    w.atoms.positions = all_positions[ts_temp.frame]
with mda.Writer("nojump.xtc", w.atoms.n_atoms) as W:
    for ts_temp in w.trajectory:
        for res, ires in zip(w.residues, np.arange(w.residues.n_residues)):
            masses = res.atoms.masses
            positions = res.atoms.positions
            com = [np.sum(positions[:,0]*masses), np.sum(positions[:,1]*masses), np.sum(positions[:,2]*masses)]/np.sum(masses)
            if ts_temp.frame > 0:
                delta_com = com - previous_com[ires]
            previous_com[ires] = com
        W.write(w)

/home/simon/.local/lib/python3.12/site-packages/MDAnalysis/coordinates/XDR.py:240: UserWarning: Reload offsets from trajectory
 ctime or size or n_atoms did not match
  warnings.warn("Reload offsets from trajectory\n "
